In [3]:
# testing model response
from openai import OpenAI

client = OpenAI(
    base_url="http://192.168.1.108:1234/v1",
    api_key="lm-studio"
)

response = client.chat.completions.create(
    model="google/gemma-3-4b",
    messages=[{"role": "user", "content": "Say hello in one short sentence."}],
)

print(response.choices[0].message.content)

ChatCompletion(id='chatcmpl-h5qr7oiycc60dzqeo6raaqp', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello there! 😊', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[]))], created=1767472911, model='google/gemma-3-4b', object='chat.completion', service_tier=None, system_fingerprint='google/gemma-3-4b', usage=CompletionUsage(completion_tokens=5, prompt_tokens=16, total_tokens=21, completion_tokens_details=None, prompt_tokens_details=None), stats={})
Hello there! 😊


In [5]:
from openai import OpenAI
from docx import Document
import pandas as pd
import os
import json
import re
import time

# -----------------------------
# LLM Client Setup
# -----------------------------
client = OpenAI(
    base_url="http://192.168.1.108:1234/v1",
    api_key="lm-studio"
)

model_name = "google/gemma-3-4b"

# -----------------------------
# File Paths
# -----------------------------
INPUT_FILE = "750-tweets-without answer.xlsx"
OUTPUT_FILE = "Gemma3_zero-shot.xlsx"


# -----------------------------
# JSON Schema for classification
# -----------------------------
classification_schema = {
    "type": "json_schema",
    "json_schema": {
        "name": "AgeismClassification",
        "schema": {
            "type": "object",
            "properties": {
                "classification": {
                    "type": "string",
                    "enum": ["Not relevant", "Ageist", "Age-positive"]
                },
                "reasoning": {
                    "type": "string",
                    "description": "Brief explanation justifying the classification decision"
                }
            },
            "required": ["classification", "reasoning"]
        },
    }
}


# -----------------------------
# Load input Excel file
# -----------------------------
df = pd.read_excel(INPUT_FILE, dtype=str)
df["classification"] = ""   # empty column to fill later
df["reasoning"] = ""

# Load keyword list
keywords_doc = Document("Keywords list_final.docx")
keywords_text = "\n".join([p.text for p in keywords_doc.paragraphs])

# -----------------------------
# Loop through tweets
# -----------------------------
for i, row in df.iterrows():
    start_time = time.time()
    text = row["text"]  # make sure your tweet text column is called "text"

    prompt_text = f"""
    You are an expert in social psychology and research on age beliefs. 

    Our goal is to create 3 lists of tweets.  To create these 3 lists, first identify all tweets that describe an older person or older people in general using Keyword List A from the attached document.
    
    Once you have identified the tweets describing older persons, classify them into one of 3 categories: (1) ageist tweets using Keyword List B and the definition below; (2) age-positive tweets using Keyword List C and the corresponding definition below; or 3) not relevant tweets. 
    
    The final dataset should include the following columns for each tweet: a single classification label (Ageist, Age-positive, or Not relevant) and reasoning explaining why the classification was made (e.g., “is ageist because uses age stereotype of frailty” or “is age-positive because praises older adults for resilience”).
    
    Our definition of “ageist” tweets are tweets that are related to older persons and show an unfavorable portrayal and/or treatment of older persons.
    Our definition of “age-positive” tweets are those that are related to older persons and show the favorable portrayal and/or treatment of older persons.
    Our definition of “not relevant” tweets are those that are not related to older persons and/ or are related to older persons but describes them in a neutral way that is not ageist or age-positive.
    
    When analyzing and classifying tweets, explicitly pay close attention to sarcasm, humor, contextual ambiguity, cultural references, figurative language, and nuanced sentiment. Tweets may use irony, exaggeration, slang, or culturally specific expressions that change their meaning. Do not rely solely on keywords; always interpret the intended meaning within context before classifying.
    
    When classifying a tweet as related to older persons, please note the following rules. Please do not consider a tweet related to an older person when: 
    (1) tweets describe old objects (e.g., “old car,” “old house”) unless the term refers to an old body part (e.g., “old knees” or “old brain”). 
    (2) “old,” refers to “old + testament” or old refers to something that is not an older person, like "the old you," "the old we," "old time," "old adage," or "old outhouse"  
    (3) “senior(s)” refers to high school or college seniors. 
    (4) “pop” refers to soda, a verb, music, food (“pop tart”), or could refer to a younger person (e.g., “my pop” could mean a young “dad”). 
    (5) For “elder” refers to Elder Scrolls (the video game). 
    (6) For “granny,” refers to granny smith which is an apple. 
    (7) For “antique” and “ancient,” describes objects rather than people. 
    (8) For “nursing home,” “assisted living,” or “care home,” does not refer to older persons.
    (9) the keyword “elder” is only referring to someone who is older than someone else, such as an “elder sibling,”  but is not referring to an older person.
    
    In addition to the rules listed above, please apply the following clarification rules:
    (1) Tweets that mention dementia, Alzheimer’s disease, memory loss, cognitive decline, or similar conditions may be classified as ageist, even if the tone appears sympathetic. This because the stereotype that all older persons develop dementia is a common negative age stereotype 
    (2) Tweets that portray an older adult in a negative, mocking, complaining, or burdensome way (such as criticizing their cooking, speech, driving, abilities, slowness, confusion, or behavior) should be classified as ageist, even if the tweet is casual or humorous. For example, “my grandma’s cooking smells terrible” or “grandpa won’t stop repeating himself” are ageist.
    (3) Tweets that express positive affection, respect, gratitude, pride, love, or admiration for an older adult should be classified as age-positive, including when someone says they miss or cherish an older relative who has passed away. This type of emotional remembrance about older persons counts as age-positive.
    (4) Tweets that describe celebrating an older person, including celebrating a birthday, should be counted as age-positive.
    (5) The word “retired” should only be coded as aging-related when it describes a person retiring due to age, not when it is used jokingly (e.g., “I’m retired from dating”).
    (6) The term “middle-aged” does not count as older adulthood and should not be classified as aging-related.
    (7) When terms like “grandma,” “grandpa,” or “old man” are used metaphorically to describe the speaker’s behavior in an ambiguous way (e.g., staying in and going to bed early), code as not relevant; However, if is clear the behavior is thought of in a negative way, code as ageist. 
    (8) tweets that infantilize older persons or treat them like babies or small children should be labeled as ageist.
    
    Also, please see the correct classifications of the following 8 tweets that could assist as you decide how to classify tweets: 
    1. "Everybody grumpy today"
       - Keywords: "grumpy" (negative age stereotype)
       - Classification: Not relevant to ageist or age-positive list (as it does not specifically refer to older persons)
     
    2. "Y'all clamoring for the days of the Old Testament y'all would be slaves in those days"
       - Keywords: "Old Testament" (irrelevant as it refers to the Bible, not older people)
       - Classification: Not relevant to ageist or age-positive list
     
    3. "I lost my granny early in life but I cherish the moments I had with her forever"
       - Keywords: "granny"
       - Classification: Age-positive (demonstrates value and positive memory of an older person)
      
    4. "WHAT?! You missed Grandparents Storytime last week? Guess what? You have a chance to catch it again on Saturday, March 21, at 10:30 a.m. Miss Tracey will sing songs, have craft time and share wonderful stories for…"
       - Keywords: "Grandparents"
       - Classification: Age-positive (promotes engagement and positive role of grandparents)
     
    5. "@captaintommoore has died from #COVID19. The 100-year-old helped raise almost £33m ($57M Cdn.) for the NHS. He was knighted last year. The Army veteran won the hearts of millions by walking 100 laps in his garden before his 100th birthday last April. #Hero #Positivity #Smile"
       - Keywords: "100-year-old", "Hero", "Positivity"
       - Classification: Age-positive (highlights achievements and positive contributions of an older person)
     
    6. "@Hichestan7 @SecPompeo @mikepompeo Still drinking from Trump’s septic tank? Its not good for you. The lies & bullshit eat away at your brain, accelerating dementia."
       - Keywords: "dementia" (negative age stereotype)
       - Classification: Ageist (uses dementia derogatorily)
     
    7. "My old man bladder is so thankful! Not a lot more stressful than worrying if you can hold it the whole movie or not!"
       - Keywords: "old man bladder" (negative age reference)
       - Classification: Ageist (implies bladder problem due to old age in a negative way)
     
    8. "Hawk Nation tune-in Sun, 4/19 on #Knightline as Coach Derek Smith (@chowan6464) discusses the recent UCF verbal commitment by Hawks speedy senior WR @davismallinger1 Live from 10-11:00 am with coach’s segment airing 10:20 am. ESPN Orlando 580 AM Hosted by @AP_Knightline https://t.co/yVmMX2jTyn"
       - Keywords: "senior"
       - Classification: Not relevant to ageist or age-positive list (senior refers to high school senior, not older person)
    
    9.  “I pop up on ya block.”
      - Keywords: "pop"
     - Classification: Not relevant because pop used as a verb so tweet not about an older person. 

    keyword list: {keywords_text}

    Post: {text}
    """

    # LLM call
    response = client.chat.completions.create(
        model=model_name,
        messages=[{"role": "user", "content": [{"type": "text", "text": prompt_text.strip()}]}],
        response_format=classification_schema,
        )

    raw = response.choices[0].message.content.strip()
    #print(f"🏷️ LLM output raw: {raw}")

    # Parse JSON safely
    try:
        parsed = json.loads(raw) if isinstance(raw, str) else raw
        classification = parsed.get("classification", "Not relevant")
        reasoning = parsed.get("reasoning", "")
    except Exception:
        classification = "Not relevant"
        reasoning = ""
    
    df.at[i, "classification"] = classification
    df.at[i, "reasoning"] = reasoning
    
    print(f"\n📄 Tweet {i+1} of {len(df)}")
    print(f"🏷️ Classification: {classification}")
    print(f"💬 Reasoning: {reasoning}")
    #print(f"⏱️ Runtime: {round(time.time() - start_time, 2)} seconds")

# -----------------------------
# Save output
# -----------------------------
df.to_excel(OUTPUT_FILE, index=False)
print(f"\n✅ Done. Saved to {OUTPUT_FILE}")


📄 Tweet 1 of 750
🏷️ Classification: Not relevant
💬 Reasoning: The tweet is a general statement about Apple products and doesn't relate to older people or any age-related stereotypes.

📄 Tweet 2 of 750
🏷️ Classification: Age-positive
💬 Reasoning: The tweet expresses a desire to borrow bingo dabbers from their grandmother, demonstrating affection and appreciation for an older family member. It’s a positive interaction centered around a loving relationship.

📄 Tweet 3 of 750
🏷️ Classification: Not relevant
💬 Reasoning: The tweet describes a job, not an older person. The emojis are irrelevant to the content.

📄 Tweet 4 of 750
🏷️ Classification: Not relevant
💬 Reasoning: The tweet uses ‘pops’ as a verb (to pop something up), not referring to an older person. It's simply describing the action of appearing online.

📄 Tweet 5 of 750
🏷️ Classification: Not relevant
💬 Reasoning: The tweet uses ‘pop’ as a verb and does not refer to an older person. It describes a specific action unrelated to age

In [6]:
import pandas as pd

# -----------------------------
# Load files
# -----------------------------
pred_df = pd.read_excel("Gemma3_zero-shot.xlsx")
ans_df  = pd.read_excel("750-tweets-answer.xlsx")

# Column names
pred_col = "classification"
aging_col = "aging related (yes/ no)"
ageist_col = "ageist (yes/ no)"
pos_col = "age positive (yes/ no)"

# -----------------------------
# Step 1: Create gold label using your rules
# -----------------------------
def create_gold_label(row):
    aging = row[aging_col].strip().upper()
    ageist = row[ageist_col].strip().upper()
    agepos = row[pos_col].strip().upper()

    if aging == "Y" and ageist == "Y":
        return "Ageist"
    elif aging == "Y" and agepos == "Y":
        return "Age-positive"
    else:
        return "Not relevant"

ans_df["gold_label"] = ans_df.apply(create_gold_label, axis=1)

# -----------------------------
# Step 2: Clean predicted label formatting
# -----------------------------
pred_df["pred_label"] = (
    pred_df[pred_col]
    .astype(str)
    .str.strip()
    .str.replace("-", "")       # e.g., "Age-positive" vs "Age positive"
    .str.title()                # normalize casing
)

# Normalize known values
pred_df["pred_label"] = pred_df["pred_label"].replace({
    "Agepositive": "Age-positive",
    "Ageist": "Ageist",
    "Not Relevant": "Not relevant",
})

# -----------------------------
# Step 3: Compare predicted label vs. gold label
# -----------------------------
gold = ans_df["gold_label"].to_numpy()
pred = pred_df["pred_label"].to_numpy()

pred_df["correct"] = pred == gold

accuracy = pred_df["correct"].mean()

print("\n====================================")
print(f"Accuracy: {accuracy:.2%}")
print(f"Correct predictions: {pred_df['correct'].sum()} / {len(pred_df)}")
print("====================================\n")

# Save output
pred_df.to_excel("Gemma3_old_accuracy_output.xlsx", index=False)


Accuracy: 61.20%
Correct predictions: 459 / 750

